In [1]:
import mediapipe as mp
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import numpy as np

In [2]:
data_path = "DATASETS/FYPD"
SEQ_LEN = 10

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic(static_image_mode=False, min_detection_confidence=0.7, min_tracking_confidence=0.5)

mp_draw = mp.solutions.drawing_utils

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [3]:
classes = os.listdir(data_path)
if '.DS_Store' in classes:
    classes.remove('.DS_Store')  
classes

['Fall Down',
 'Sit down',
 'Walking',
 'Lying Down',
 'Stand up',
 'Standing',
 'Sitting',
 'Chest Pain',
 'Coughing']

In [7]:
def draw(image, results):
    spec = mp_draw.DrawingSpec(color=(255, 255, 255), thickness=0, circle_radius=0)
    mp_draw.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, spec)
    mp_draw.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, spec)
    mp_draw.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, spec)
    mp_draw.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, spec)

    

def extract_landmarks(path, seq_len):
    valid = True
    video_reader = cv2.VideoCapture(path)
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

    if int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT)) < seq_len:
        valid = False
    # print(int(video_reader.get(cv2.CAP_PROP_ FRAME_COUNT)))
    skip_frames_window = max(int(video_frames_count/seq_len), 1)
    landmark_list = []
    image_list = []
 
    for frame_counter in range(seq_len):
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)

        success, frame = video_reader.read() 
        if not success:
            break

        resized_frame = cv2.resize(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB), (480, 480))
        
        results = pose.process(resized_frame)

        pose_lm = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten()  if results.pose_landmarks else np.zeros(132)
        # face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
        # rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(63)
        # lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(63)
        
        # lm_list = np.concatenate([pose, face, lh, rh])

        # mp_draw.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, spec)

        # draw(resized_frame, results)
    

        # image_list.append(resized_frame)
        landmark_list.append(pose_lm)
    
    video_reader.release()

    return np.array(landmark_list), valid#, image_list

In [10]:
class_num =6
vid_num = 19
frame_num = 1

tes_vid = data_path + '/' + classes[class_num] + '/' + os.listdir(data_path + '/' + classes[class_num])[vid_num]
lm_list, valid = extract_landmarks(tes_vid, 10)


print(lm_list[frame_num].shape)
# plt.imshow(img_list[frame_num])
print(valid)
print(len(lm_list))

(132,)
True
10


In [101]:
def extract_landmarks(path, seq_len):
    valid = True
    video_reader = cv2.VideoCapture(path)
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

    if int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT)) < seq_len:
        valid = False
    # print(int(video_reader.get(cv2.CAP_PROP_ FRAME_COUNT)))
    skip_frames_window = max(int(video_frames_count/seq_len), 1)
    landmark_list = []
    # image_list = []
 
    for frame_counter in range(seq_len):
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)

        success, frame = video_reader.read() 
        if not success:
            break

        resized_frame = cv2.resize(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB), (480, 480))
        
        results = holistic.process(resized_frame)

        pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten()  if results.pose_landmarks else np.zeros(132)
        face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
        rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(63)
        lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(63)
        
        lm_list = np.concatenate([pose, face, lh, rh])

        # draw(resized_frame, results)
    

        # image_list.append(resized_frame)
        landmark_list.append(lm_list)
    
    video_reader.release()

    return np.array(landmark_list), valid #, image_list

In [11]:
def get_data(path, class_num, seq_len=SEQ_LEN):
    data = []

    vid_list = os.listdir(path)

    if '.DS_Store' in vid_list:
        vid_list.remove('.DS_Store') 

    for vid in vid_list:
        vid_path = os.path.join(path, vid)
        
        lm, valid = extract_landmarks(vid_path, seq_len)
    
        if valid:
            data.append(lm)
    
    labels = np.ones((len(data)))*class_num
    print(f'Total Videos: {len(vid_list)}, Extracted Videos: {len(data)}')
    return np.array(data), labels

In [20]:
class_num = 0
fall = get_data(f"{data_path}/{classes[class_num]}", class_num)

Total Videos: 632, Extracted Videos: 556


In [19]:
class_num = 1
sit = get_data(f"{data_path}/{classes[class_num]}", class_num)

Total Videos: 265, Extracted Videos: 261


In [18]:
class_num = 2
walk = get_data(f"{data_path}/{classes[class_num]}", class_num)

Total Videos: 950, Extracted Videos: 930


In [17]:
class_num = 3
lie = get_data(f"{data_path}/{classes[class_num]}", class_num)

Total Videos: 543, Extracted Videos: 489


In [16]:
class_num = 4
stand_up = get_data(f"{data_path}/{classes[class_num]}", class_num)

Total Videos: 422, Extracted Videos: 410


In [15]:
class_num = 5
standing = get_data(f"{data_path}/{classes[class_num]}", class_num)

Total Videos: 808, Extracted Videos: 698


In [14]:
class_num = 6
sitting = get_data(f"{data_path}/{classes[class_num]}", class_num)

Total Videos: 450, Extracted Videos: 432


In [13]:
class_num = 7
chest = get_data(f"{data_path}/{classes[class_num]}", class_num)

Total Videos: 58, Extracted Videos: 58


In [12]:
class_num = 8
cough = get_data(f"{data_path}/{classes[class_num]}", class_num)

Total Videos: 37, Extracted Videos: 37


In [104]:
cough[0].shape, cough[1].shape

((37, 10, 1662), (37,))

In [21]:
X = np.concatenate([fall[0], sit[0], walk[0], lie[0], stand_up[0], standing[0], sitting[0], chest[0], cough[0]], axis=0)
y = np.concatenate([fall[1], sit[1], walk[1], lie[1], stand_up[1], standing[1], sitting[1], chest[1], cough[1]], axis=0)

In [22]:
y =  np.expand_dims(y, -1)

In [23]:
X.shape, y.shape

((3871, 10, 132), (3871, 1))

In [24]:
np.save(f'AI/data/X_final_seq_{SEQ_LEN}.npy', X)
np.save(f'AI/data/y_final_seq_{SEQ_LEN}.npy', y)